In [1]:
import requests
import pandas as pd
from config import geoapify_key

In [2]:
base_url = f"https://api.geoapify.com/v1/geocode/reverse?"

params = {
    "type": "postcode",
    "apiKey": geoapify_key
}

In [3]:
citi_bike_data_1 = pd.read_csv('resources/202301-citibike-tripdata.csv',
                               dtype={'start_station_id': 'str', 'end_station_id': 'str'})
citi_bike_data_2 = pd.read_csv('resources/202302-citibike-tripdata.csv',
                               dtype={'start_station_id': 'str', 'end_station_id': 'str'})
combo_df = citi_bike_data_1.append(citi_bike_data_2, ignore_index=True)
citi_bike_data_1 = None
citi_bike_data_2 = None

In [4]:
citi_bike_data_3 = pd.read_csv('resources/202303-citibike-tripdata.csv',
                               dtype={'start_station_id': 'str', 'end_station_id': 'str'})
combo_df = combo_df.append(citi_bike_data_3, ignore_index=True)
citi_bike_data_3 = None

In [6]:
combo_df.to_csv('resources/2023-Q1_CitiBike.csv', index=False)

In [7]:
station_id_group = combo_df.groupby('start_station_id')[['start_lat', 'start_lng']].mean()
station_id_group = station_id_group.reset_index()

In [8]:
for index, row in station_id_group.iterrows():
    start_lat = combo_df.loc[index, "start_lat"]
    start_lng = combo_df.loc[index, "start_lng"]
    params["lat"] = start_lat
    params["lon"] = start_lng

    results = requests.get(base_url, params=params)
    results = results.json()

    station_id_group.loc[index, "city"] = results["features"][0]["properties"]["city"]
    station_id_group.loc[index, "state"] = results["features"][0]["properties"]["state"]
    station_id_group.loc[index, "postcode"] = results["features"][0]["properties"]["postcode"]

In [9]:
station_id_group.to_csv("resources/station_details.csv", index=False)